In [3]:
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from gen_ai_hub.proxy.langchain.openai import ChatOpenAI
from gen_ai_hub.proxy.core.proxy_clients import get_proxy_client
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv
import os
from langchain.memory import ConversationBufferMemory

load_dotenv()

os.environ["AICORE_HOME"] = "D:/Anubhav_GetAI_Batch/profiles"
os.environ["AICORE_PROFILE"] = "anubhav"
dep_id = os.getenv("LLM_DEPLOYMENT_ID")

proxy_client = get_proxy_client('gen-ai-hub')

llm = ChatOpenAI(proxy_model_name='gpt-35-turbo', proxy_client=proxy_client, deployment_id=dep_id)

llm.invoke("when was last eruption in mt. fuji?")

AIMessage(content='The last eruption of Mt. Fuji occurred in 1707 during the Hoei eruption. Since then, the volcano has been dormant.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 17, 'total_tokens': 44, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gpt-3.5-turbo-1106', 'system_fingerprint': 'fp_0165350fbb', 'id': 'chatcmpl-C96mvQz47Z87E4LV4bUP90T61bMhH', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--3eb3ff3b-0e17-4b2b-88eb-a688385e8132-0', usage_metadata={'input_tokens': 17, 'output_tokens': 27, 'total_tokens': 44, 'input_token_details': {}, 'output_token_details': {}})

In [4]:
test_post='''📢 Attention Gurgaon residents! ⚠️

Can we talk about the pathetic state of our roads and water flooding for a moment? 🗑️🤢 
It seems like the local authorities have forgotten about our beloved public area on Sector 65 Road. 
Seriously, has anyone seen the piles of rubbish and litter scattered everywhere which causes massive water flooding when rain comes? 
🚯 It's like a swimming pool next to our doorstep! I mean, who needs to fix broken roads, blocked man holes, do you agree?

📍 Sector 65 Road, Gurgaon

It's mildly infuriating how we pay our taxes and yet we have to put up with this filth! 🤬 I'm not asking for Presidential Palace-like cleanliness, but a basic level of hygiene wouldn't hurt. 
Hopefully, the authorities will wake up from their slumber and do something about it ASAP. Let's keep our fingers crossed! 🤞

#CleanUpYourAct #Sector65RoadNightmare #GurgaonFloods

'''

In [5]:
info_dict={ 
"category": 
'''Classify the post in one of the following categories: \"PUBLIC CLEANLINESS\", \"ROADS & FOOTPATHS\", \
\"FACILITY & PARK MAINTENANCE\", \"PESTS\", \"DRAINS & SEWERS\".
If none of the categories fits, return \"OTHER\".''',                 

"priority": 
'''Identify the priority to be given to the reported issues into \"4-Low\", \"3-Medium\", \"2-High\", \"1-Very High\". .
    4-Low : the issue does not pose any problem with public safety and does not necessarily need to be handled urgently. 
    3-Medium : the issue does not cause any immediate danger, but it has significant and negative impact on the daily life of people in the neighborhood.
    2-High : the issue needs to be resolved quickly because it can potentially cause dangerous situations or disruptions. 
    1-Very High : the issue needs to be handled as soon as possible, as it is a matter of public safety. 
    ''',          
            
"summary": 
"Summarize the reported issue in 40 characters and a neutral tone.",
            
"description": 
"Summarize the reported issue in not more that 300 characters and a neutral tone.",

"address":
"Extract the address where the issue is taking place. Return the street only and omit the town or country",
            
"location": 
"Extract the coordinates where the issue has been notices. The format should be: float, float",
       
"sentiment" : 
'''Classify the sentiment of the post into \"NEUTRAL\", \"NEGATIVE\", \"VERY NEGATIVE\"
1. NEUTRAL: if the post reports an issue politely, in a calm tone
2. NEGATIVE: if the post shows irony, impatience, annoyance
3. VERY NEGATIVE: the post is rude or it expresses rage, hatred towards the public authority
'''
}

In [6]:
from langchain.prompts import ChatPromptTemplate

template=''' Extract information from the social media post delimited by triple backticks.
```{post}``` 
'''

post_prompt= ChatPromptTemplate.from_template(template)

In [7]:
functions = [
    {
        "name": "post_analysis",
        "summary": "Extract information from the social media post",
        "parameters": {
            "type": "object",
            "properties": {
                
                "category": {"type": "string", "description": info_dict["category"], 
                             "enum":['PUBLIC CLEANLINESS','ROADS & FOOTPATHS','FACILITY & PARK MAINTENANCE',
                                     'PESTS', 'DRAINS & SEWERS','OTHER']},
                "priority": {"type": "string", "description": info_dict["priority"], 
                             "enum": ['4-Low','3-Medium', '2-High', '1-Very High'] },
                "summary": {"type": "string", "description": info_dict["summary"]},
                "description": {"type": "string", "description": info_dict["description"]},
                "address": {"type": "string", "description": info_dict["address"]},
                "location": {"type": "string", "description": info_dict["location"]},
                "sentiment": {"type": "string", "description": info_dict["sentiment"],
                              "enum": ['NEUTRAL','NEGATIVE', 'VERY NEGATIVE']},
            },
            "required": ["category", "priority"],
        },
    }
]

In [8]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

chain = (
    post_prompt
    | llm.bind(function_call={"name": "post_analysis"}, functions=functions)
    | JsonOutputFunctionsParser()
)

chain.invoke({"post": test_post})

{'category': 'ROADS & FOOTPATHS',
 'priority': '2-High',
 'summary': 'Pathetic state of roads and water flooding',
 'description': 'The roads are in a pathetic state with piles of rubbish and litter scattered everywhere, causing massive water flooding during rain.',
 'address': 'Sector 65 Road',
 'location': '28.4294, 77.0738',
 'sentiment': 'NEGATIVE'}